#  Estimating a Dynamic Discrete Choice Model for Bus Engine Replacement: An Application of Hotz & Miller’s Conditional Choice Probability (CCP) Method

We will use ChatGPT (o3-mini-high) and prompt engineering to implement the estimation of a dynamic discrete choice model for bus engine replacement, following Hotz & Miller's (1993) Conditional Choice Probability (CCP) method. The implementation will be built from scratch, leveraging ChatGPT for code generation, debugging, and optimization

**Key Assumption**: The observed data on actions and states $\{d_{it}, x_{it}\}$ are genertaed from the optimal decision rule. 

## Task 1: Estimate the CCP Nonparametrically
$$
\hat{P}(d = 1 \mid x) =
\frac{\sum_{i,t} \mathbf{1} \{ d_{it} = 1, x_{it} = x \} }
     {\sum_{i,t} \mathbf{1} \{ x_{it} = x \} },
$$

**Initial prompt**: As in John Rust 1987 paper, a data set recording the engine replacement decisions in a month (d_it); x_it is the bus mileage discretized to N bins; give me a python code to implement the conditional choice probability estimation. Data set is pandas df with columns of mileage and replacement decisions; first use a cut-off say 5000 miles to discretize the state space, and then implement the CCP estimation as in the equation.  

**Improved prompt by ChatGPT**: 

As in John Rust (1987), we have a dataset recording monthly bus engine replacement decisions (`d_it`), where `x_it` represents the bus mileage, discretized into N intervals. The dataset is stored as a Pandas DataFrame with the following columns:

- `mileage`: Continuous mileage of the bus.
- `replacement`: Binary indicator (1 = engine replaced, 0 = not replaced).

Task:
1. Discretize the mileage state space** using a 5,000-mile cutoff (i.e., group mileage into bins of width 5,000 miles).
2. Estimate the Conditional Choice Probability (CCP) for engine replacement, using the empirical probability formula:

   $$
   \hat{P}(d = 1 \mid x) = \frac{\sum_{i,t} 1\{d_{it} = 1, x_{it} = x\}}{\sum_{i,t} 1\{x_{it} = x\}}
   $$

3. Implement this in Python, and return a DataFrame containing:
   - Discretized mileage bins
   - Estimated CCPs for each mileage bin
   - Number of observations per bin

In [2]:
import pandas as pd
import numpy as np

def estimate_ccp(df, mileage_col='mileage', decision_col='replacement', bin_size=5000):
    """
    Estimate the Conditional Choice Probability (CCP) of replacement for each
    discretized mileage state using a bin_size cutoff (default = 5000 miles).

    Parameters
    ----------
    df : pd.DataFrame
        Must contain columns for mileage and replacement decision.
    mileage_col : str
        Column name for mileage.
    decision_col : str
        Column name for the binary replacement decision (0 or 1).
    bin_size : int
        Mileage interval width (default 5000). E.g., 0-4999, 5000-9999, etc.

    Returns
    -------
    pd.DataFrame
        A DataFrame with:
        - 'mileage_bin': the discretized mileage interval (e.g., multiples of bin_size).
        - 'CCP': the fraction of times d=1 in each mileage_bin.
        - 'count': how many (i,t) observations fall in that bin.
    """

    # 1. Discretize mileage: group into intervals of size 'bin_size'
    df = df.copy()
    df['mileage_bin'] = (df[mileage_col] // bin_size) * bin_size

    # 2. Group by the discretized mileage_bin
    grouped = df.groupby('mileage_bin')

    # 3. Compute CCP: fraction of replacements (d=1) for each bin
    ccp = grouped[decision_col].mean()   # P(d=1 | x)

    # 4. Also get the count of observations in each bin
    counts = grouped[decision_col].count()

    # 5. Combine into a result DataFrame
    result = pd.DataFrame({
        'mileage_bin': ccp.index,
        'CCP': ccp.values,
        'count': counts.values
    }).reset_index(drop=True)

    return result

# -------------------------------------------------------------------------
# Example usage with a mock dataset:
if __name__ == "__main__":
    # Create a random dataset of 1000 observations:
    np.random.seed(123)
    df_example = pd.DataFrame({
        'mileage': np.random.randint(0, 200_000, 1000),  # random mileage up to 200k
        'replacement': np.random.choice([0,1], size=1000, p=[0.8, 0.2])  # ~20% replacements
    })

    # Estimate CCP with 5,000-mile bins
    ccp_df = estimate_ccp(df_example, bin_size=5000)



In [3]:
ccp_df.head()

,mileage_bin,CCP,count
0,0,0.172414,29
1,5000,0.206897,29
2,10000,0.086957,23
3,15000,0.172414,29
4,20000,0.093750,32


## Task 2: Estimate the transition probability of state non-parametrically

$$  P(\Delta x \in \text{Category}) = \frac{\text{Count of observations in category}}{\text{Total observations}}
$$

In this example, the distribution of $\Delta x_{it} := x_{it+1} - x_{it}$ is independent with decision. Replacement decision only resets $x_{it}$. 

**Initial Prompt**: The monthly increment in bus mileage falls into one of three categories: below 5,000 miles, between 5,000 and 10,000 miles, and above 10,000 miles.Estimate the parameters of a multinomial distribution using the same data. The dataset has a panel structure, grouped by bus ID. First, sort the data by bus ID and month. Then, compute the monthly mileage increment using first-order differencing. Finally, estimate the multinomial probabilities by counting occurrences within each of the three mileage increment categories.     

**Improved Prompt by ChatGPT**: 

The monthly increment in mileage for a bus falls into one of three categories:
Below 5,000 miles
Between 5,000 and 10,000 miles
Above 10,000 miles

Task:

Data Structure:

The dataset is in panel format, grouped by bus ID.
Each bus has multiple monthly mileage records.

Preprocessing:

Sort the data by bus ID and month.

Compute monthly mileage increments using first-order differencing.

Parameter Estimation:

Count the occurrences of mileage increments falling into each of the three categories.

Estimate parameters of a multinomial distribution by computing the relative frequency of each category.

Expected Output:

A table with three probabilities corresponding to the likelihood of each mileage increment category.



In [6]:
import pandas as pd
import numpy as np

def estimate_multinomial_mileage(df, bus_id_col='bus_id', mileage_col='mileage', month_col='month'):
    """
    Estimate the multinomial distribution parameters for bus mileage increments.

    Parameters
    ----------
    df : pd.DataFrame
        Panel data containing bus mileage records over time.
    bus_id_col : str
        Column name for bus ID.
    mileage_col : str
        Column name for mileage.
    month_col : str
        Column name for month or time indicator.

    Returns
    -------
    pd.DataFrame
        A DataFrame with probabilities for each mileage increment category.
    """

    # 1. Sort data by bus ID and month
    df = df.sort_values(by=[bus_id_col, month_col]).copy()

    # 2. Compute first-order differencing (monthly mileage increment)
    df['mileage_increment'] = df.groupby(bus_id_col)[mileage_col].diff()

    # 3. Categorize mileage increments into three bins
    df['increment_category'] = pd.cut(df['mileage_increment'], 
                                      bins=[-np.inf, 5000, 10000, np.inf], 
                                      labels=['Below 5000', '5000-10000', 'Above 10000'])

    # 4. Count occurrences of each category
    category_counts = df['increment_category'].value_counts(normalize=True)

    # 5. Convert to DataFrame for output
    result = category_counts.reset_index()
    result.columns = ['Mileage Increment Category', 'Probability']

    return result

# -------------------------------------------------------------------------
# Example usage with a mock dataset:
if __name__ == "__main__":
    # Create a synthetic dataset
    np.random.seed(42)
    df_example = pd.DataFrame({
        'bus_id': np.repeat(np.arange(1, 51), 12),  # 50 buses, each with 12 months
        'month': list(range(1, 13)) * 50,
        'mileage': np.random.randint(0, 200_000, 600)  # Random mileage values
    })

    # Estimate multinomial probabilities
    multinomial_params = estimate_multinomial_mileage(df_example)


## Task 3: Simulate a dynamic path of $(d,x)$ using the estimated CCP $P(d \mid x)$ and the trasition probability $F(x' \mid x, d)$

Keep in mind $x' = x + \Delta x$, where $\Delta x$ follows a stationary Multinomial distribution with three cases.  

**Initial Promt**: Starting from a tuple of engine replecemnet decision (1 or 0) and bus mileage $(d, x)$, simulate a stochastic path of the tuple given the estimated CCP $P(d \mid x)$ and the process $x' = x + \Delta x$, where  $\Delta x \sim Multinomial(\cdot ; p, q, (1-p-q)) $. $x=0$ if $d=1$. The estimates of $P(d \mid x)$ are saved as a data frame including mileage bins and frequency within each of the bins.  The estimates of $\Delta x \sim Multinomial(\cdot ; p, q, (1-p-q)) $ are saved as a data frame with three cases along with corresponding probabilities. A python code simulating a path of the tuple starting from $(d,x)$. 

**Imrpoved Prompt**: 

Starting from an **initial tuple** $(d, x)$, where:
- $d$ is the **engine replacement decision** (1 = replace, 0 = continue).
- $x$ is the **bus mileage**.

Simulate a **stochastic path** of the tuple $(d_t, x_t)$ over time, given the following:

**1. Conditional Choice Probability (CCP) Estimates**
- The probability of engine replacement, $P(d = 1 \mid x)$, is provided in a **DataFrame** with:
  - **Mileage Bins**: Discretized mileage states.
  - **Estimated CCP**: The probability of replacement for each mileage bin.

**2. Mileage Transition Process**
- The next mileage state follows the transition rule:

  $$
  x' =
  \begin{cases}
  \Delta x, & \text{if } d = 1 \text{ (replacement occurs)}, \\
  x + \Delta x, & \text{otherwise}.
  \end{cases}
  $$

- The mileage increment $\Delta x$ follows a **Multinomial distribution**:

  $$
  \Delta x \sim \text{Multinomial}(\cdot ; p, q, 1 - p - q)
  $$

- The estimated parameters $(p, q, 1 - p - q)$ are saved in a **DataFrame**, where:
  - The three cases correspond to mileage increments:
    - **Below 5,000 miles** ($\Delta x_1$)
    - **Between 5,000 and 10,000 miles** ($\Delta x_2$)
    - **Above 10,000 miles** ($\Delta x_3$)
  - The DataFrame includes **probabilities for each mileage increment case**.

**3. Simulation Task**
Write a **Python function** that:

1. **Takes as input:**
   - An **initial state** $(d_0, x_0)$.
   - The **CCP DataFrame** for $P(d \mid x)$.
   - The **Multinomial Mileage Transition DataFrame**.
   - A specified number of periods $T$ for the simulation.

2. **Simulates a stochastic sequence** of $(d_t, x_t)$ over $T$ periods using:
   - A **random draw** from $P(d \mid x)$ to determine whether to replace the engine.
   - A **random draw** from the **Multinomial distribution** to determine mileage increments when $d = 0$.

3. **Outputs:**
   - A time series of simulated $(d_t, x_t)$ values.



In [1]:
import numpy as np
import pandas as pd

def get_ccp(x, ccp_df, bin_col='mileage_bin', ccp_col='CCP', bin_size=5000):
    """
    Given a continuous mileage x, find the bin in ccp_df and return CCP = P(d=1|x).
    
    Parameters
    ----------
    x : float
        Current mileage.
    ccp_df : pd.DataFrame
        Must contain columns [bin_col, ccp_col], 
        where bin_col is the bin start (e.g., 0, 5000, 10000, etc.) 
        and ccp_col is the probability of replacement for that bin.
    bin_col : str
        Name of the column that indicates the start of the mileage bin.
    ccp_col : str
        Name of the column that indicates the CCP (prob of d=1).
    bin_size : int
        Width of each mileage bin (e.g., 5000).
    
    Returns
    -------
    float
        CCP = P(d=1|x).
    """
    # Find the bin index by integer division
    bin_start = (int(x) // bin_size) * bin_size
    
    # Locate matching row in ccp_df
    row = ccp_df.loc[ccp_df[bin_col] == bin_start]
    if not row.empty:
        return float(row[ccp_col].values[0])
    else:
        # If x exceeds any defined bin, use the highest bin or fallback
        # Here we pick the nearest largest bin for demonstration
        max_bin_start = ccp_df[bin_col].max()
        if x >= max_bin_start:
            row_max = ccp_df.loc[ccp_df[bin_col] == max_bin_start]
            return float(row_max[ccp_col].values[0])
        else:
            # For safety, default 0 if not found
            return 0.0

def draw_delta_x(multidf):
    """
    Draw a mileage increment from three possible categories,
    given a DataFrame with categories and probabilities.
    
    The DataFrame (multidf) should have columns:
      'category' in { 'below_5k', '5k_to_10k', 'above_10k' }
      'prob' which sums to 1
    We then pick one category via np.random.choice,
    and sample a random integer within that category's range.
    
    Returns
    -------
    int
        A random mileage increment from one of the three categories.
    """
    categories = multidf['category'].values
    probs = multidf['prob'].values
    
    chosen_cat = np.random.choice(categories, p=probs)
    
    if chosen_cat == 'below_5k':
        return np.random.randint(0, 5000)
    elif chosen_cat == '5k_to_10k':
        return np.random.randint(5000, 10000)
    else:
        # 'above_10k'
        return np.random.randint(10000, 20000)  # arbitrary upper bound, can be adjusted

def simulate_path(d0, x0, T, ccp_df, multidf):
    """
    Simulate a path (d_t, x_t) for T periods.
    
    Parameters
    ----------
    d0 : int
        Initial engine decision (0 or 1).
    x0 : float
        Initial mileage.
    T : int
        Number of periods to simulate.
    ccp_df : pd.DataFrame
        DataFrame containing mileage bins and CCP = P(d=1|x).
        Columns: ['mileage_bin', 'CCP', ...].
    multidf : pd.DataFrame
        DataFrame containing the probabilities for the three increment categories.
        Columns: ['category', 'prob'] with sum(prob)=1.
    
    Returns
    -------
    pd.DataFrame
        A DataFrame with columns ['period', 'd_t', 'x_t'] describing the simulated path.
    """
    # Initialize
    d_current = d0
    x_current = x0
    
    records = []
    
    for t in range(T):
        # Record (d_t, x_t)
        records.append({'period': t, 'd_t': d_current, 'x_t': x_current})
        
        # 1. Draw next d based on x_current
        ccp_val = get_ccp(x_current, ccp_df)
        d_next = 1 if (np.random.rand() < ccp_val) else 0
        
        # 2. Draw mileage increment
        delta_x = draw_delta_x(multidf)
        
        # 3. Update x
        if d_next == 1:
            # x' = Delta x (reset + new mileage)
            x_next = delta_x
        else:
            # x' = x + Delta x
            x_next = x_current + delta_x
        
        # Prepare for next iteration
        d_current = d_next
        x_current = x_next
    
    # Convert to DataFrame
    sim_df = pd.DataFrame(records)
    return sim_df

# -------------------------------------------------------------------------
# Example usage
if __name__ == "__main__":
    # Mock CCP DataFrame: bins of 5000 up to 20000
    ccp_data = {
        'mileage_bin': [0, 5000, 10000, 15000],
        'CCP': [0.1, 0.15, 0.2, 0.25]  # example probabilities
    }
    ccp_df = pd.DataFrame(ccp_data)
    
    # Mock Multinomial DataFrame:
    # category -> { below_5k, 5k_to_10k, above_10k }
    # prob -> sum = 1
    multi_data = {
        'category': ['below_5k', '5k_to_10k', 'above_10k'],
        'prob': [0.4, 0.3, 0.3]
    }
    multidf = pd.DataFrame(multi_data)
    
    # Simulate for T=10 periods starting from (d0=0, x0=2000)
    sim_result = simulate_path(d0=0, x0=2000, T=10, ccp_df=ccp_df, multidf=multidf)
    print(sim_result)


   period  d_t    x_t
0       0    0   2000
1       1    0  11906
2       2    0  20564
3       3    0  28536
4       4    1   4879
5       5    0  22924
6       6    0  37549
7       7    0  51190
8       8    0  51514
9       9    0  61201


## Task 4. Approximate the choice-specific value function via simulation  

**Initial Prompt**: Based on the previous code of simulating a stochastic path of action and state, prepare a python code of evaluting the choice-specific value function on $S$ simulated paths starting from $(d,x)$ and then take the averare as in the equation. In the code, $u(x,d,\theta)$ is defined separately and parsed in as an argument; the CCP $\hat{P}(d \mid x)$ is a data frame with mileage bins and corresponding frequencies; $\gamma$ is the Euler constant.  
$$
\tilde{V}(x, d; \theta) \approx \frac{1}{S} \sum_{s} \Bigg[ u(x, d; \theta) + \gamma - \log \big(\hat{P}(d \mid x)\big) \\
\quad + \beta \Big[ u(x^{\prime s}, d^{\prime s}; \theta) + \gamma - \log \big(\hat{P}(d^{\prime s} \mid x^{\prime s})\big) \\
\quad + \beta \Big[ u(x^{\prime\prime s}, d^{\prime\prime s}; \theta) + \gamma - \log \big(\hat{P}(d^{\prime\prime s} \mid x^{\prime\prime s})\big) + \beta \cdots \Big] \Big] \Bigg]
$$

**Improved Prompt by ChatGPT**: A Python function to evaluate the choice-specific value function $\tilde{V}(x, d; \theta)$ using $S$ simulated paths. The approximation follows:

$$
\tilde{V}(x, d; \theta) \approx \frac{1}{S} \sum_{s} \Bigg[ u(x, d; \theta) + \gamma - \log \big(\hat{P}(d \mid x)\big) 
+ \beta \Big[ u(x^{\prime s}, d^{\prime s}; \theta) + \gamma - \log \big(\hat{P}(d^{\prime s} \mid x^{\prime s})\big) 
+ \beta \Big[ u(x^{\prime\prime s}, d^{\prime\prime s}; \theta) + \gamma - \log \big(\hat{P}(d^{\prime\prime s} \mid x^{\prime\prime s})\big) + \beta \cdots \Big] \Big] \Bigg]
$$

#### Requirements

The function should take as input:
- An initial state $(d, x)$.
- A function $u(x, d, \theta)$ that computes the immediate utility.
- A CCP DataFrame $\hat{P}(d \mid x)$ containing mileage bins and their corresponding estimated choice probabilities.
- A Multinomial Mileage Transition DataFrame defining possible mileage increments and their probabilities.
- A discount factor $\beta$.
- The number of simulated paths $S$.
- The number of time periods $T$ for forward simulation.
- A parameter $\theta$ that affects the utility function.

#### Methodology

- **Stochastic Simulation**:
  - The function simulates $S$ stochastic sequences of $(d_t, x_t)$ over $T$ periods.
  - At each step, the engine replacement decision $d$ is drawn based on the CCP.
  - If $d = 1$, mileage resets to zero; otherwise, a mileage increment is sampled from the multinomial distribution.

- **Recursive Value Computation**:
  - The recursive sum of discounted utilities along each path is computed:
  
    $$
    V_s = \sum_{t=0}^{T} \beta^t \Big[ u(x_t, d_t; \theta) + \gamma - \log \big(\hat{P}(d_t \mid x_t)\big) \Big]
    $$

  - The final estimate is obtained by averaging over all simulated paths:

    $$
    \tilde{V}(x, d; \theta) = \frac{1}{S} \sum_{s=1}^{S} V_s
    $$

- **Implementation Details**:
  - The Euler constant $\gamma \approx 0.5772$ is used.
  - The function ensures that the mileage transitions follow the multinomial process.
  - The choice probabilities from the CCP DataFrame determine the engine replacement decision.



In [5]:
import numpy as np
import pandas as pd

def evaluate_choice_value(initial_state, utility_fn, ccp_df, transition_df, beta, theta, S, T, random_seed=None):
    """
    Evaluate the choice-specific value function $\tilde{V}(x, d; \theta)$ using $S$ simulated paths.
    
    Parameters:
    -----------
    initial_state : tuple
        The initial state as (d, x), where d is the engine replacement decision (1 = replace, 0 = continue)
        and x is the current mileage.
    utility_fn : function
        A function u(x, d, theta) that computes the immediate utility.
    ccp_df : pd.DataFrame
        DataFrame containing mileage bins and corresponding estimated choice probabilities. It should have columns:
            - 'mileage_lower'
            - 'mileage_upper'
            - 'ccp': probability of replacement, i.e., $\hat{P}(1 \mid x)$.
              (The probability of not replacing is given by $1 - \text{ccp}$.)
    transition_df : pd.DataFrame
        DataFrame defining the possible mileage increments and their probabilities. It should have columns:
            - 'mileage_increment'
            - 'probability'
    beta : float
        Discount factor.
    theta : any
        Parameter(s) that affect the utility function.
    S : int
        Number of simulated paths.
    T : int
        Number of time periods for forward simulation.
    random_seed : int, optional
        Seed for reproducibility.
    
    Returns:
    --------
    float
        The estimated choice-specific value function $\tilde{V}(x, d; \theta)$.
    """
    if random_seed is not None:
        np.random.seed(random_seed)
    
    gamma = 0.5772  # Euler constant
    total_value = 0.0

    for s in range(S):
        # Initialize state for this simulation
        d, x = initial_state
        path_value = 0.0
        discount = 1.0
        
        for t in range(T):
            # Determine the CCP probability for the current mileage x
            ccp_row = ccp_df[(ccp_df['mileage_lower'] <= x) & (x < ccp_df['mileage_upper'])]
            if not ccp_row.empty:
                p_replace = ccp_row.iloc[0]['ccp']
            else:
                p_replace = 0.0  # Default if x is out of range
            
            # Simulate decision based on the CCP
            if np.random.rand() < p_replace:
                d = 1
            else:
                d = 0
            
            # Get the probability of the chosen decision
            prob_chosen = p_replace if d == 1 else (1 - p_replace)
            # Avoid log(0)
            log_prob = np.log(prob_chosen) if prob_chosen > 0 else 0
            
            # Compute the immediate contribution for this period
            immediate = utility_fn(x, d, theta) + gamma - log_prob
            path_value += discount * immediate
            discount *= beta  # Update discount factor
            
            # Update state for next period
            if d == 1:
                # Engine replacement: reset mileage to 0
                x = 0
            else:
                # Sample mileage increment from the multinomial distribution
                increments = transition_df['mileage_increment'].values
                probs = transition_df['probability'].values
                x += np.random.choice(increments, p=probs)
        
        total_value += path_value
    
    return total_value / S

# Example usage:
def example_utility(x, d, theta):
    # Example utility function: linear in mileage and decision cost
    return theta['a'] * x - theta['b'] * d

if __name__ == "__main__":
    # Define the CCP DataFrame with mileage bins and replacement probabilities
    ccp_data = {
        'mileage_lower': [0, 5000, 10000],
        'mileage_upper': [5000, 10000, 15000],
        'ccp': [0.05, 0.1, 0.2]
    }
    ccp_df = pd.DataFrame(ccp_data)
    
    # Define the Transition DataFrame for mileage increments
    transition_data = {
        'mileage_increment': [2500, 7500, 12500],
        'probability': [0.6, 0.3, 0.1]
    }
    transition_df = pd.DataFrame(transition_data)
    
    # Initial state: (d, x)
    initial_state = (0, 3000)
    beta = 0.95
    theta = {'a': 0.01, 'b': 5}
    S = 1000  # Number of simulated paths
    T = 20    # Number of time periods per path
    
    estimated_value = evaluate_choice_value(initial_state, example_utility, ccp_df, transition_df, beta, theta, S, T, random_seed=42)
    print("Estimated Choice-Specific Value Function:", estimated_value)

Estimated Choice-Specific Value Function: 4941.948666204341


## Task 5: Define a Non-linear Least Squares Estimator for Parameter Training

**Initial Prompt**: On the initial panel data of bus mileage and repacement decisions $\{d_{it}, x_{it}\}$, a python code to define the objective fucntion of the following minimization problem 
$$ \bar{\theta} = \arg\min_{\theta} \Big\| \hat{\delta}(x) - 
    \Big[ \hat{V}(x, d = 1; \theta) - \hat{V}(x, d = 0; \theta) \Big] \Big\|.
$$

where $\hat{\delta}(x) := \log \hat{p}(d = 1 \mid x) - \log \hat{p}(d = 0 \mid x)$. For each data point, creating the mileage bin corresponding to $x$, and the compute $\hat{\delta}(x)$ using previous results of CCP, which is a data frame with mileage bins and corresponding frequencies. The differene in value functions is calculated using the previous code.  

**Improved Prompt by ChatGPT**: We have panel data on bus mileage and replacement decisions, given by $\{d_{it}, x_{it}\}$. Your task is to write Python code that defines the objective function for the following minimization problem:

$$
\bar{\theta} = \arg\min_{\theta} \Big\| \hat{\delta}(x) - \Big[ \hat{V}(x, d=1; \theta) - \hat{V}(x, d=0; \theta) \Big] \Big\|
$$

where:
- $\hat{\delta}(x) := \log \hat{p}(d=1 \mid x) - \log \hat{p}(d=0 \mid x)$.
- $\hat{p}(d \mid x)$ are estimated choice probabilities obtained from a CCP DataFrame. This DataFrame contains mileage bins and the corresponding frequencies (or probabilities) of replacement decisions.
- $\hat{V}(x, d; \theta)$ are the choice-specific value functions, computed using the simulation code previously provided. In that code, the value function is evaluated by simulating forward $T$ periods along $S$ paths.

#### Detailed Instructions:
1. **Mileage Binning**:  
   For each data point $x$ in the panel data, determine its corresponding mileage bin using the CCP DataFrame.

2. **Compute $\hat{\delta}(x)$**:  
   Using the frequencies in the CCP DataFrame, compute:
   $$
   \hat{\delta}(x) = \log \hat{p}(d=1 \mid x) - \log \hat{p}(d=0 \mid x)
   $$
   where $\hat{p}(d=1 \mid x)$ is the frequency (or estimated probability) of replacement, and $\hat{p}(d=0 \mid x)$ is $1 - \hat{p}(d=1 \mid x)$.

3. **Value Function Difference**:  
   Use the previously defined simulation code to compute the choice-specific value functions $\hat{V}(x, d=1; \theta)$ and $\hat{V}(x, d=0; \theta)$ for each data point.

4. **Objective Function**:  
   The objective function measures the difference between the computed value function difference and the observed log odds $\hat{\delta}(x)$:
   $$
   \text{Objective}(\theta) = \Big\| \hat{\delta}(x) - \Big[ \hat{V}(x, d=1; \theta) - \hat{V}(x, d=0; \theta) \Big] \Big\|
   $$
   This objective should be computed for every data point, and then aggregated (for example, by summing or taking the mean across observations) to form a single loss value that can be minimized with respect to \(\theta\).

5. **Minimization Setup**:  
   Write the Python code to define this objective function so that it can be used in a minimization routine (e.g., using `scipy.optimize.minimize`).

#### Summary of Required Inputs:
- **Panel Data**: $\{d_{it}, x_{it}\}$
- **CCP DataFrame**: Contains mileage bins (with lower and upper bounds) and frequencies of replacement decisions.
- **Transition DataFrame**: Used for simulating future mileage increments.
- **Simulation Code**: For evaluating $\hat{V}(x, d; \theta)$ given a state and a decision.
- **Discount Factor $\beta$**.
- **Number of Simulated Paths $S$** and **Time Periods $T$**.
- **Parameter $\theta$** affecting the utility function $u(x, d, \theta)$.

In [2]:
import numpy as np
import pandas as pd

def evaluate_choice_value(initial_state, utility_fn, ccp_df, transition_df, beta, theta, S, T, random_seed=None):
    """
    Evaluate the choice-specific value function \hat{V}(x, d; \theta) using S simulated paths.
    
    Parameters:
        initial_state (tuple): (d, x), where d is the decision (1 for replacement, 0 otherwise)
                               and x is the current mileage.
        utility_fn (function): A function u(x, d, theta) returning immediate utility.
        ccp_df (pd.DataFrame): DataFrame with mileage bins and corresponding CCP values.
                               Expected columns: 'mileage_lower', 'mileage_upper', 'ccp'.
        transition_df (pd.DataFrame): DataFrame with mileage increments and their probabilities.
                                      Expected columns: 'mileage_increment', 'probability'.
        beta (float): Discount factor.
        theta (dict): Parameters for the utility function.
        S (int): Number of simulated paths.
        T (int): Number of time periods for simulation.
        random_seed (int, optional): Seed for reproducibility.
    
    Returns:
        float: The estimated choice-specific value function \hat{V}(x, d; \theta).
    """
    if random_seed is not None:
        np.random.seed(random_seed)
    
    gamma = 0.5772  # Euler constant
    total_value = 0.0

    for s in range(S):
        d, x = initial_state
        path_value = 0.0
        discount = 1.0
        
        for t in range(T):
            # Find the CCP probability for the current mileage x using the mileage bins
            ccp_row = ccp_df[(ccp_df['mileage_lower'] <= x) & (x < ccp_df['mileage_upper'])]
            if not ccp_row.empty:
                p_replace = ccp_row.iloc[0]['ccp']
            else:
                p_replace = 0.0  # default if x does not fall into any bin
            
            # Simulate decision based on CCP
            if np.random.rand() < p_replace:
                d = 1
            else:
                d = 0
            
            # Get the probability of the chosen decision
            prob_chosen = p_replace if d == 1 else (1 - p_replace)
            log_prob = np.log(prob_chosen) if prob_chosen > 0 else 0
            
            # Immediate contribution: utility + Euler constant - log(probability)
            immediate = utility_fn(x, d, theta) + gamma - log_prob
            path_value += discount * immediate
            discount *= beta
            
            # Update mileage: if replaced, reset to 0; else add a mileage increment
            if d == 1:
                x = 0
            else:
                increments = transition_df['mileage_increment'].values
                probs = transition_df['probability'].values
                x += np.random.choice(increments, p=probs)
        
        total_value += path_value
    
    return total_value / S

def objective(theta, panel_data, ccp_df, transition_df, beta, utility_fn, S, T, random_seed=None):
    """
    Define the objective function for the minimization problem:
    
    $$ \bar{\\theta} = \\arg\\min_{\\theta} \\Big\\| \\hat{\\delta}(x) - 
    \\Big[ \\hat{V}(x, d=1; \\theta) - \\hat{V}(x, d=0; \\theta) \\Big] \\Big\\| $$
    
    where 
    $$ \\hat{\\delta}(x) = \\log \\hat{p}(d=1 \\mid x) - \\log \\hat{p}(d=0 \\mid x). $$
    
    Parameters:
        theta (dict): Parameters for the utility function.
        panel_data (pd.DataFrame): Panel data containing bus mileage. Expected column: 'x'.
        ccp_df (pd.DataFrame): DataFrame with mileage bins and CCP values.
                               Columns: 'mileage_lower', 'mileage_upper', 'ccp'.
        transition_df (pd.DataFrame): DataFrame with mileage increments and probabilities.
                                      Columns: 'mileage_increment', 'probability'.
        beta (float): Discount factor.
        utility_fn (function): Function u(x, d, theta) that returns immediate utility.
        S (int): Number of simulated paths for value function evaluation.
        T (int): Number of time periods for simulation.
        random_seed (int, optional): Seed for reproducibility.
    
    Returns:
        float: The aggregated squared error over the panel data.
    """
    total_error = 0.0
    n = len(panel_data)
    
    for idx, row in panel_data.iterrows():
        x = row['x']
        # Identify the mileage bin for x in the CCP DataFrame
        bin_mask = (ccp_df['mileage_lower'] <= x) & (x < ccp_df['mileage_upper'])
        bin_data = ccp_df[bin_mask]
        if not bin_data.empty:
            p_replace = bin_data.iloc[0]['ccp']
        else:
            p_replace = 0.05  # default probability if x is out of range
        
        p_not_replace = 1 - p_replace
        # Compute the observed log odds difference
        delta_hat = np.log(p_replace) - np.log(p_not_replace)
        
        # Evaluate the choice-specific value functions for both decisions
        V1 = evaluate_choice_value((1, x), utility_fn, ccp_df, transition_df, beta, theta, S, T, random_seed)
        V0 = evaluate_choice_value((0, x), utility_fn, ccp_df, transition_df, beta, theta, S, T, random_seed)
        value_diff = V1 - V0
        
        # Compute squared error for this data point
        error = delta_hat - value_diff
        total_error += error**2
    
    return total_error / n

# Example usage:
def example_utility(x, d, theta):
    # A sample utility function: linear in mileage and decision cost.
    return theta['a'] * x - theta['b'] * d

if __name__ == "__main__":
    # Sample panel data: bus mileage values
    panel_data = pd.DataFrame({
        'x': [3000, 7000, 12000]  # example mileage observations
    })
    
    # CCP DataFrame: define mileage bins and estimated replacement probabilities
    ccp_data = {
        'mileage_lower': [0, 5000, 10000],
        'mileage_upper': [5000, 10000, 15000],
        'ccp': [0.05, 0.1, 0.2]
    }
    ccp_df = pd.DataFrame(ccp_data)
    
    # Transition DataFrame: define mileage increments and their probabilities
    transition_data = {
        'mileage_increment': [2500, 7500, 12500],
        'probability': [0.6, 0.3, 0.1]
    }
    transition_df = pd.DataFrame(transition_data)
    
    beta = 0.95
    S = 100  # number of simulated paths
    T = 20   # number of time periods per simulation
    theta_init = {'a': 0.01, 'b': 5}  # initial parameter guess
    
    # Compute the objective function value for the current theta
    obj_value = objective(theta_init, panel_data, ccp_df, transition_df, beta, example_utility, S, T, random_seed=42)
    print("Objective Function Value:", obj_value)

Objective Function Value: 5.139776266985947


## Task 5: Define a Class to Ensemble Different Pieces of Code 

In [6]:
import pandas as pd
import numpy as np

class HotzMillerBusEngine(object):

    def __init__(self, path, file, tperiods=1000, discount=0.995):
        self.T = tperiods
        self.B = discount
        self.df = self._load_data(path, file)

    @staticmethod    
    def _load_data(path, file):
        return pd.read_csv(os.path.join(path, file), sep='\s+', header=0)
    
    def estimate_ccp(df, mileage_col='mileage', decision_col='replacement', bin_size=5000):
        """
        Estimate the Conditional Choice Probability (CCP) of replacement for each
        discretized mileage state using a bin_size cutoff (default = 5000 miles).
    
        Parameters
        ----------
        df : pd.DataFrame
            Must contain columns for mileage and replacement decision.
        mileage_col : str
            Column name for mileage.
        decision_col : str
            Column name for the binary replacement decision (0 or 1).
        bin_size : int
            Mileage interval width (default 5000). E.g., 0-4999, 5000-9999, etc.
    
        Returns
        -------
        pd.DataFrame
            A DataFrame with:
            - 'mileage_bin': the discretized mileage interval (e.g., multiples of bin_size).
            - 'CCP': the fraction of times d=1 in each mileage_bin.
            - 'count': how many (i,t) observations fall in that bin.
        """
    
        # 1. Discretize mileage: group into intervals of size 'bin_size'
        df = df.copy()
        df['mileage_bin'] = (df[mileage_col] // bin_size) * bin_size
    
        # 2. Group by the discretized mileage_bin
        grouped = df.groupby('mileage_bin')
    
        # 3. Compute CCP: fraction of replacements (d=1) for each bin
        ccp = grouped[decision_col].mean()   # P(d=1 | x)
    
        # 4. Also get the count of observations in each bin
        counts = grouped[decision_col].count()
    
        # 5. Combine into a result DataFrame
        result = pd.DataFrame({
            'mileage_bin': ccp.index,
            'CCP': ccp.values,
            'count': counts.values
        }).reset_index(drop=True)
    
        return result

    
